In [ ]:
#
# Universidad EAFIT 
# 2025-2
# SI7016 - NLP - Lecture 07
#

In [ ]:
# instalar dependencias
%pip install openai langchain langchain_community streamlit
%pip install faiss-cpu
# instalar ChromaDB como retriever 
%pip install chromadb

In [ ]:
# paso 1:
import os
from langchain.chat_models import ChatOpenAI

# Configurar la API Key (puedes establecerla como variable de entorno)
#os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# bash: export OPENAI_API_KEY="your-openai-api-key"

# Inicializar el modelo GPT-4
llm = ChatOpenAI(model_name="gpt-4")


In [ ]:
# Inicializar la Base de Datos Vectorial

import chromadb

# Inicializar la base de datos ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Crear una colección en ChromaDB
collection = chroma_client.get_or_create_collection(name="chatbot_knowledge")

#


In [ ]:
# Agregar Datos de Referencia a la Base de Datos

documents = [
    {"id": "1", "text": "La inteligencia artificial es el campo de la informática que estudia cómo crear sistemas capaces de realizar tareas que requieren inteligencia humana."},
    {"id": "2", "text": "GPT-4 es un modelo de lenguaje de OpenAI basado en la arquitectura de transformers."},
    {"id": "3", "text": "LangChain es una biblioteca para desarrollar aplicaciones basadas en modelos de lenguaje."}
]

# Agregar documentos a ChromaDB
for doc in documents:
    collection.add(ids=[doc["id"]], documents=[doc["text"]])


In [ ]:
# Crear un Retriever con ChromaDB

from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Inicializar embeddings con OpenAI
embeddings = OpenAIEmbeddings()

# Crear el VectorStore con ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# Crear el retriever para buscar documentos relevantes
retriever = vectorstore.as_retriever()


In [ ]:
# paso 2

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Crear memoria de la conversación
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Integrar memoria en la cadena conversacional

#chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory)

# fix con un retriever en chromadb
chat_chain = ConversationalRetrievalChain.from_llm(llm, memory=memory,retriever=retriever)


In [ ]:
# Paso 3: Construir la Función del Chatbot

def chat_with_gpt(user_input):
    response = chat_chain({"question": user_input})
    return response["answer"]


In [ ]:
# Ejecución del Chatbot
#%pip install streamlit
#%pip install tiktoken

!streamlit run app.py


In [ ]:
# mejoras propuestas

# Agregar RAG (Retrieval-Augmented Generation) con bases de datos vectoriales y tus propios datos.
# Integrar API externas (ejemplo: Wikipedia, bases de datos de conocimiento).
# Incluir Voice-to-Text para hacer un chatbot por voz.
